In [ ]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import time


In [ ]:
start = time.time()
targetSite = 'https://series.naver.com/novel/top100List.series'
driver = webdriver.Chrome()
driver.get(targetSite)
wait = WebDriverWait(driver, 10)

# 각 탭 (실시간, 일간, 주간, 월간 Top 100) 크롤링
for i in range(1, 5, 1):  # 실시간, 일간, 주간, 월간
    driver.find_element(By.XPATH, f'//*[@id="container"]/div[1]/ul/li[{i}]/a').click()
    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div/ul/li[1]/div[2]/h3/a')))
    
    Titles = []  # 제목 저장할 리스트
    Genre = []  # 장르 저장할 리스트
    Author = []  # 작가 저장할 리스트
    Start = []  # 연재 시작일
    Last = []  # 마지막 연재일
    Count = []  # 연재수
    Good = []  # 추천수
    Point = []  # 평점
    Comment = []  # 댓글 수
    
    for p in range(1, 6, 1):
        if p != 1:  # 첫 페이지가 아닌 경우 페이지 이동
            driver.find_element(By.XPATH, f'//*[@id="content"]/div/div[2]/a[{p-1}]').click()
            wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div/ul/li[1]/div[2]/h3/a')))
        
        for j in range(1, 21, 1):
            driver.find_element(By.XPATH, f'//*[@id="content"]/div/ul/li[{j}]/div[2]/h3/a').click()
            
            if "login" in driver.current_url or "adult" in driver.current_url: # 19금, 로그인 필요한 경우
                # 이후 전처리 할 떄 쉽게 하도록 19금 통일
                Titles.append("19금")
                Genre.append("19금")
                Author.append("19금")
                Point.append("19금")
                Start.append("19금")
                Last.append("19금")
                Count.append("19금")
                Good.append("19금")
                Comment.append("19금")
                driver.back()
                continue
            
            # 연재 시작일과 마지막 연재일 수집
            # 정렬 기준이 랜덤이라 Start, Last에 들어가는게 반대로 들어가는 경우가 있음 (최신순, 1화부터)
            a = driver.find_element(By.XPATH, '//*[@id="content"]/table/thead/tr/th[1]/div')  # 최신순
            b = driver.find_element(By.XPATH, '//*[@id="content"]/table/thead/tr/th[2]/div')  # 1화부터
            
            if a.get_attribute('class') == 'on _changeTicketSortOrder(DESC)':  # 최신순 정렬
                wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em')))
                Last.append(driver.find_element(By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em').text)
                b.click()
                wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em')))
                Start.append(driver.find_element(By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em').text)
                driver.back()
            
            elif b.get_attribute('class') == 'on _changeTicketSortOrder(ASC)':  # 1화부터 정렬
                wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em')))
                Start.append(driver.find_element(By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em').text)
                a.click()
                wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em')))
                Last.append(driver.find_element(By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em').text)
                driver.back()
            
            try:
                Titles.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/h2').text)
                Good.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[2]/ul/li[2]/div/a/em').text)
            except:
                Titles.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/h2').text)
                Good.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[2]/ul/li[2]/div/a/em').text)
            
            try:
                Point.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[1]/em').text)
            except:
                Point.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/em').text)
            
            Genre.append(driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[2]/span/a').text)
            Author.append(driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[3]/a').text)
            Count.append(driver.find_element(By.XPATH, '//*[@id="content"]/h5/strong').text)
            
            wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="cbox_module"]/div/div[1]/span')))
            Comment.append(driver.find_element(By.XPATH, '//*[@id="cbox_module"]/div/div[1]/span').text)
            
            driver.back()
    
    df = pd.DataFrame({"제목": Titles, "장르": Genre, "작가": Author, "연재 시작일": Start, "마지막 연재일": Last, "회수": Count, "댓글수": Comment, "추천수": Good, "평점": Point})
    df.index = df.index + 1
    
    if i == 1:
        df.to_csv('./data/NSeries_Now.txt')
    elif i == 2:
        df.to_csv('./data/NSeries_Today.txt')
    elif i == 3:
        df.to_csv('./data/NSeries_Week.txt')
    else:
        df.to_csv('./data/NSeries_Month.txt')
driver.quit()
end = time.time()

In [ ]:
dfN=pd.read_csv('./data/NSeries_Now100.txt', index_col=0)
dfT=pd.read_csv('./data/NSeries_Today100.txt', index_col=0)
dfW=pd.read_csv('./data/NSeries_Week100.txt', index_col=0)
dfM=pd.read_csv('./data/NSeries_Month100.txt', index_col=0)
print(end-start)

In [ ]:
dfN

In [ ]:
dfT

In [ ]:
dfW

In [ ]:
dfM